- code to extract data from the RT task
- preprocess them: rename rows with dots in the name, remove incomplete/redundant subs, impossibly small RTs, add a column for inconsistent during-after responses
- save the data



In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

os.chdir('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/RT_online_task')
working_dir = os.getcwd()
print('working dir:',working_dir)

working dir: /Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/RT_online_task


In [2]:
data_folder = r'/Users/f0053cz/Documents/HCP_decision_task/data'
os.listdir(data_folder)

['60fce0e8ed92418cabcf0f9c_expt_v3_2021-07-25_09h36.55.282.log.gz',
 '60ede2f9e9aced79841374bf_expt_v3_2021-07-25_14h45.44.459.log.gz',
 '60fd5d01137be0173165dcfe_expt_v3_2021-07-25_08h39.36.916.csv',
 '60fc5c5723636d2e4e6acd1b_expt_v3_2021-07-25_14h56.09.793.csv',
 '60fcd84f537cf56255168da7_expt_v3_2021-07-25_09h54.48.232.csv',
 '60fd22f7fd87f2c9d63a3639_expt_v3_2021-07-25_14h37.25.903.csv',
 '5b7d70ff1e35f000017319e5_expt_v3_2021-07-25_14h54.35.263.log.gz',
 '60fd5eb480090422254ed2e6_expt_v3_2021-07-25_09h53.36.170.log.gz',
 '60f935e186acc8fa59c26a26_expt_v3_2021-07-25_09h00.56.262.log.gz',
 '60fd2e10d0e4aa8935d07269_expt_v3_2021-07-25_14h54.05.867.csv',
 '60fd03302b29a3efada39165_expt_v3_2021-07-25_08h52.20.361.csv',
 '60fce0e8ed92418cabcf0f9c_expt_v3_2021-07-25_09h36.55.282.csv',
 '60fd3627553a78d22e703535_expt_v3_2021-07-25_14h51.08.713.csv',
 '60fce65c600566fd3c2c050f_expt_v3_2021-07-25_09h52.37.514.csv',
 '60fd5c3031886d6cd578e6c9_expt_v3_2021-07-25_14h40.49.399.csv',
 '60fd20e2

In [3]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [4]:
print('num all files:',len(os.listdir(data_folder)))
csv_files=[f for f in os.listdir(data_folder) if '.csv' in f]
print('num csv files:',len(csv_files))

num all files: 215
num csv files: 112


In [5]:
movie_names = ['COAXINGB0','BILLIARDA1','DRIFTINGA2','FISHING3','RANDOMMECH4',
              'SCARING5', 'SEDUCINGB6', 'STARA7', 'SURPRISINGB8', 'TENNISA9']

In [6]:
own_id = '60f5c61782ba68550b8f2cae'
len(own_id)

24

In [7]:
# make a list of csv file names
from os import listdir

def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if ((filename.endswith( suffix )) & \
            (filename[:11] !='PARTICIPANT') & (filename[:24] !='60f5c61782ba68550b8f2cae'))]

filenames = find_csv_filenames(data_folder) # list of csv files in a folder
len(filenames)
#for name in filenames: # prints csv filenames
#    print(name)

108

In [8]:
# print out feedback from subjects
dat_fb = []
for name in filenames:
    try:
        if '2021-07-25' in name:
            dat = pd.read_csv(os.path.join(data_folder,name))
            if dat.shape[0] > 11:
                mat = dat.loc[:,'feedback_text'].values
                mat = mat[-1]
                #print(mat)
                dat_fb.append(mat)

            else:
                print('incomplete sub:', dat['participant'].values[0])
    except:
        print('empty file')
fb = [i for i in dat_fb if len(i)>3]
for i in fb:
    print(i)
print(f'\nfeedback from {len(fb)} people')

incomplete sub: 60fd4227a0da73a530f9c5a7
empty file
empty file
incomplete sub: 60fcd0211918af2f6707ea9d
empty file
empty file
A: this was an interesting excersise
A: wish i had more than  seconds to answer afterwards
A: i felt as if the triangles lived in the square and whenever they interacted it was as if one triangle was kicking the other out of the house or inviting the other inside
A: slightly more instruction
A: was fun
A: this made me question the nature of social interaction entirely,  not sure i know what it  is anymore
A: good animations not a long survey
A: none
A: all was great,  except the last question part.  i wish i could go back and change my answer on one question about the gender 
A: numbering all the questions would be nice to see how many there are
A: perhaps each video numbered to indicate how far into the exercise you are. 
A: i think this survey is actually really good
A: good
A: quite fun
A: long pauses are distracting
A: fun
A: would only allow me to describe 

In [9]:
# print columns in the raw data
print(pd.read_csv(os.path.join(data_folder,name)).columns)

Index(['cons_key.keys', 'cons_key.corr', 'cons_key.rt', 'distr_off_key.keys',
       'distr_off_key.rt', 'demoRespDur.keys', 'demoRespDur.rt',
       'respAfter.keys', 'respAfter.rt', 'demo_loop.thisRepN',
       'demo_loop.thisTrialN', 'demo_loop.thisN', 'demo_loop.thisIndex',
       'demo_loop.ran', 'fileName', 'participant', 'session', 'date',
       'expName', 'psychopyVersion', 'OS', 'frameRate', 'study_id',
       'respDur.keys', 'respDur.rt', 'task_loop.thisRepN',
       'task_loop.thisTrialN', 'task_loop.thisN', 'task_loop.thisIndex',
       'task_loop.ran', 'racialCat_clicked', 'hispanicCat_clicked',
       'genderCat_clicked', 'ageKeys.keys', 'ageKeys.rt', 'age_input',
       'fbKeys.keys', 'fbKeys.rt', 'feedback_text', 'currLocKeys.keys',
       'currLocKeys.rt', 'currLoc_text', 'prolificID_keys.keys',
       'prolificID_keys.rt', 'prolific_id', 'thank_you_key.keys',
       'thank_you_key.rt'],
      dtype='object')


In [10]:
def extract_rows_cols(data):
    # function to extract certain rows and columns (as needed for analysis) from a single subject's data
    # as a dataframe, define a few new variables and return the new df
    
    for i,m in enumerate(data['fileName'].values):
        if movie_names[0] in str(m):
            start_row  = i
    #print(start_row)
    data_rows = data.iloc[start_row:start_row+10,:]
    data_rows['movie'] = [str(i).split('/')[1].split('.')[0] for i in data_rows['fileName'].values]
    
    movie_ind = np.empty((data_rows.shape[0],))
    for r,movie_row in enumerate(data_rows['movie']):
        for i,m in enumerate(movie_names):
            if movie_row == m:
                movie_ind[r] = i
                break
    #print(movie_ind)
    data_rows['movie_ind'] = movie_ind
    colnames = ['respDur.keys','respDur.rt', 'respAfter.keys', 'respAfter.rt','participant','movie','date','movie_ind']
    data_rows = data_rows.loc[:,colnames]
    consistency = np.zeros((data_rows.shape[0],))
    for i in range(data_rows.shape[0]):
        if data_rows['respDur.keys'].values[i] == data_rows['respAfter.keys'].values[i]:
            #same response during and after
            consistency[i] = 1  
        else:
            if data_rows['respAfter.keys'].values[i] in ['left','right']:
                consistency[i] = -1 # response flipped from left -> right or vice versa.
            elif data_rows['respAfter.keys'].values[i] == 'down':
                consistency[i] = 0.5 # response  changed to "unsure"
            elif np.isnan(data_rows['respAfter.keys'].values[i]):
                consistency[i] = 0 # response missed (possibly due to limited RT after)"
                
    data_rows['consistency'] = consistency
    return data_rows

#example to see that the code works
s = 15 #file nr
print(extract_rows_cols(pd.read_csv(os.path.join(data_folder,filenames[s]))))  
    


   respDur.keys  respDur.rt respAfter.keys  respAfter.rt  \
1          left      7.0192           left        1.1695   
2         right     15.1607           down        2.7923   
3         right     15.8680          right        1.0867   
4          left      7.9749           left        0.5625   
5          left     13.5669           left        0.5200   
6          left     17.0102           left        0.5021   
7          left     11.1045           left        0.6565   
8         right      9.1158           down        2.2266   
9          left      8.0555           left        1.3728   
10        right     11.0810          right        0.7131   

                 participant         movie                     date  \
1   60fd530ff30159b5785d34d7     COAXINGB0  2021-07-25_09h49.10.548   
2   60fd530ff30159b5785d34d7    BILLIARDA1  2021-07-25_09h49.10.548   
3   60fd530ff30159b5785d34d7    DRIFTINGA2  2021-07-25_09h49.10.548   
4   60fd530ff30159b5785d34d7      FISHING3  2021-07-25_

<ipython-input-10-cd1bc4cfe8dd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rows['movie'] = [str(i).split('/')[1].split('.')[0] for i in data_rows['fileName'].values]
<ipython-input-10-cd1bc4cfe8dd>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rows['movie_ind'] = movie_ind


In [11]:
# extract the required rows and columnsnusing the function defined above, save it as one long dataframe

inds = [0,30]#np.arange(0,len(filenames))
i = -1
for name in filenames:
    i += 1
    try:
        if '2021-07-25' in name:
            dat = pd.read_csv(os.path.join(data_folder,name))
            if dat.shape[0] > 11:
                dat = extract_rows_cols(dat)

                if i == 0:
                    dat_all = pd.DataFrame(dat, columns = dat.columns)
                else:
                    #print('else')
                    dat_all = dat_all.append(dat,ignore_index=True)
                    #print(dat_all)

            else:
                print('incomplete sub:', dat['participant'].values[0])
    except:
        print('empty file')
        
print('\nAll extracted. \n\nrows/columns: ',dat_all.shape)
dat_all.head() 

<ipython-input-10-cd1bc4cfe8dd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rows['movie'] = [str(i).split('/')[1].split('.')[0] for i in data_rows['fileName'].values]
<ipython-input-10-cd1bc4cfe8dd>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rows['movie_ind'] = movie_ind


incomplete sub: 60fd4227a0da73a530f9c5a7
empty file
empty file
incomplete sub: 60fcd0211918af2f6707ea9d
empty file
empty file

All extracted. 

rows/columns:  (1020, 9)


,respDur.keys,respDur.rt,respAfter.keys,respAfter.rt,participant,movie,date,movie_ind,consistency
0,left,3.070585,left,0.868510,60fd5d01137be0173165dcfe,COAXINGB0,2021-07-25_08h39.36.916,0.0,1.0
1,right,1.909830,right,0.669700,60fd5d01137be0173165dcfe,BILLIARDA1,2021-07-25_08h39.36.916,1.0,1.0
2,right,2.362310,right,0.497975,60fd5d01137be0173165dcfe,DRIFTINGA2,2021-07-25_08h39.36.916,2.0,1.0
3,left,4.312300,left,1.568915,60fd5d01137be0173165dcfe,FISHING3,2021-07-25_08h39.36.916,3.0,1.0
4,right,1.539055,right,0.667690,60fd5d01137be0173165dcfe,RANDOMMECH4,2021-07-25_08h39.36.916,4.0,1.0


In [12]:

sub_ids = np.unique(dat_all['participant'].values) # count unique subs
len(sub_ids)


101

In [13]:
# nr of prolific ids
prolific_approved = pd.read_excel(r'/Users/f0053cz/Documents/HCP_decision_task_others/demographics_maindata/Prolific_subs.xlsx')
prolificIDs = prolific_approved['participant_id'].values
len(prolificIDs)


101

In [14]:
# quality check 1: are the data of APPROVED subs on Prolific and saved data (who's not me) the same?
print('not on prolific:')
for i in range(len(sub_ids)):
    if sub_ids[i] not in prolificIDs: # if there's a subject who's not on Prolific
        print(sub_ids[i])
        
print('..\nnot on pavlovia')
for i in range(len(prolificIDs)):
    if prolificIDs[i] not in sub_ids: # if there's a subject who's not on Prolific
        print(prolificIDs[i])
print('..')

not on prolific:
..
not on pavlovia
..


In [15]:
#quality check 2: are there any subjects who repeat? If so, remove them

sub_ids = np.unique(dat_all['participant'].values)

for i in sub_ids:
    rows = np.where(dat_all['participant'].values==i)[0]
    if len(rows) > 10: # more than 10 rows per subject (since we only have the movie rows now)
        print(dat_all.iloc[rows,:]) # print subs who repeat
        dat_all = dat_all.loc[dat_all['participant']!=i,:]

dat_all.shape # 100 subs after removing redundancies

    respDur.keys  respDur.rt respAfter.keys  respAfter.rt  \
340         left       3.116           left         0.836   
341        right       4.668          right         0.606   
342        right       7.683          right         1.008   
343         left       6.073           left         0.787   
344        right      15.630           down         2.093   
345         left      10.206           left         0.659   
346         left       5.133           left         1.073   
347        right       5.657          right         0.692   
348         left       5.543           left         0.579   
349        right       5.623            NaN           NaN   
940         left       2.345           left         1.073   
941        right       3.549          right         0.973   
942        right       4.578          right         0.412   
943         left       5.264           left         0.960   
944         left      13.566           left         1.072   
945         left      12

(1000, 9)

In [16]:
# preproc 1: remove trials with impossibly small RTs for during or after

rows = (dat_all['respDur.rt'] > .1) | (dat_all['respAfter.rt'] > .1)
dat_all = dat_all.loc[rows,:]
dat_all.shape

(999, 9)

In [17]:
# preproc 2: remove trials with no "during" response at all - since that's our main question here

rows = (dat_all['respDur.keys'] == 'left') | (dat_all['respDur.keys'] == 'right')
dat_all = dat_all.loc[rows,:]
dat_all.shape

(994, 9)

In [26]:
# rename '.' in colnames with '_' to avoid errors while accessing columns using the dot operator

cols = []
for i in dat_all.columns:
    if '.' in i:
        cols.append(i)

cols_new = [i.replace('.','_') for i in cols]
df = pd.DataFrame({'cols':cols, 'cols_new':cols_new})
dictcol = df.set_index('cols').to_dict()['cols_new']    

dat_all = dat_all.rename(columns = dictcol)

dat_all

,respDur_keys,respDur_rt,respAfter_keys,respAfter_rt,participant,movie,date,movie_ind,consistency
0,left,3.070585,left,0.868510,60fd5d01137be0173165dcfe,COAXINGB0,2021-07-25_08h39.36.916,0.0,1.0
1,right,1.909830,right,0.669700,60fd5d01137be0173165dcfe,BILLIARDA1,2021-07-25_08h39.36.916,1.0,1.0
2,right,2.362310,right,0.497975,60fd5d01137be0173165dcfe,DRIFTINGA2,2021-07-25_08h39.36.916,2.0,1.0
3,left,4.312300,left,1.568915,60fd5d01137be0173165dcfe,FISHING3,2021-07-25_08h39.36.916,3.0,1.0
4,right,1.539055,right,0.667690,60fd5d01137be0173165dcfe,RANDOMMECH4,2021-07-25_08h39.36.916,4.0,1.0
...,...,...,...,...,...,...,...,...,...
1015,left,7.740700,left,0.696100,60fd20e2fcd0da955a2b7b3c,SCARING5,2021-07-25_16h57.42.386,5.0,1.0
1016,left,4.984500,left,0.574300,60fd20e2fcd0da955a2b7b3c,SEDUCINGB6,2021-07-25_16h57.42.386,6.0,1.0
1017,left,4.825600,NaN,NaN,60fd20e2fcd0da955a2b7b3c,STARA7,2021-07-25_16h57.42.386,7.0,0.0
1018,left,0.688400,left,1.417700,60fd20e2fcd0da955a2b7b3c,SURPRISINGB8,2021-07-25_16h57.42.386,8.0,1.0


In [27]:
# save this dataframe
final_data_folder = r'/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/RT_online_task'
dat_all.to_csv(os.path.join(final_data_folder,'RT_data_all_subs.csv'))

In [30]:
dat_all

,respDur_keys,respDur_rt,respAfter_keys,respAfter_rt,participant,movie,date,movie_ind,consistency
0,left,3.070585,left,0.868510,60fd5d01137be0173165dcfe,COAXINGB0,2021-07-25_08h39.36.916,0.0,1.0
1,right,1.909830,right,0.669700,60fd5d01137be0173165dcfe,BILLIARDA1,2021-07-25_08h39.36.916,1.0,1.0
2,right,2.362310,right,0.497975,60fd5d01137be0173165dcfe,DRIFTINGA2,2021-07-25_08h39.36.916,2.0,1.0
3,left,4.312300,left,1.568915,60fd5d01137be0173165dcfe,FISHING3,2021-07-25_08h39.36.916,3.0,1.0
4,right,1.539055,right,0.667690,60fd5d01137be0173165dcfe,RANDOMMECH4,2021-07-25_08h39.36.916,4.0,1.0
...,...,...,...,...,...,...,...,...,...
1015,left,7.740700,left,0.696100,60fd20e2fcd0da955a2b7b3c,SCARING5,2021-07-25_16h57.42.386,5.0,1.0
1016,left,4.984500,left,0.574300,60fd20e2fcd0da955a2b7b3c,SEDUCINGB6,2021-07-25_16h57.42.386,6.0,1.0
1017,left,4.825600,NaN,NaN,60fd20e2fcd0da955a2b7b3c,STARA7,2021-07-25_16h57.42.386,7.0,0.0
1018,left,0.688400,left,1.417700,60fd20e2fcd0da955a2b7b3c,SURPRISINGB8,2021-07-25_16h57.42.386,8.0,1.0
